In [20]:
from google.colab import drive
import os
import glob
import json
import pandas as pd
import chardet


In [21]:
# Монтирование Google Drive
drive.mount('/content/drive')

# Путь к папке с данными
data_folder = "/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset"

# Получаем список всех папок в каталоге данных
folders = [f for f in glob.glob(data_folder + "/*/", recursive=False)]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
print(folders)

['/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchCreativeWork/', '/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchScreeningEvent/', '/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/GetWeather/', '/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/AddToPlaylist/', '/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/RateBook/', '/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/BookRestaurant/', '/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/PlayMusic/']


In [23]:
from google.colab import drive
import os

# Монтирование Google Drive
drive.mount('/content/drive')

# Путь к папке с данными
data_folder = "/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset"

# Получаем список всех папок в каталоге данных
folders = [folder[0] for folder in os.walk(data_folder)]

print(folders)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset', '/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchCreativeWork', '/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchScreeningEvent', '/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/GetWeather', '/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/AddToPlaylist', '/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/RateBook', '/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/BookRestaurant', '/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/PlayMusic']


In [24]:
# Получаем список всех json файлов в каталоге данных
json_files = [f for f in glob.glob(data_folder + "**/*", recursive=True)]

# Выводим список всех файлов
for json_file in json_files:

    print("smth", json_file)


smth /content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchCreativeWork
smth /content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchScreeningEvent
smth /content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/GetWeather
smth /content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/AddToPlaylist
smth /content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/RateBook
smth /content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/BookRestaurant
smth /content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/PlayMusic


In [27]:
import glob

# Цикл по всем папкам
for folder in folders:
    # Получаем список всех json файлов в текущей папке
    json_files = [f for f in glob.glob(folder + "/*.json")]

    # Выводим список всех файлов в текущей папке
    print(f"В папке {folder} найдены следующие файлы:")
    for json_file in json_files:
        print(json_file)


В папке /content/drive/MyDrive/Colab Notebooks/SNIPS_dataset найдены следующие файлы:
В папке /content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchCreativeWork найдены следующие файлы:
/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchCreativeWork/train_SearchCreativeWork.json
/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchCreativeWork/validate_SearchCreativeWork.json
/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchCreativeWork/train_SearchCreativeWork_full.json
В папке /content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchScreeningEvent найдены следующие файлы:
/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchScreeningEvent/validate_SearchScreeningEvent.json
/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchScreeningEvent/train_SearchScreeningEvent_full.json
/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchScreeningEvent/train_SearchScreeningEvent.json
В папке /content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/G

In [29]:
# Создаем словари для хранения обучающих и валидационных данных
train_data = []
validate_data = []

# Цикл по всем папкам
for folder in folders:
    # Получаем список всех json файлов в текущей папке
    json_files = [f for f in glob.glob(folder + "/*.json")]

    # Цикл по всем json файлам в текущей папке
    for json_file in json_files:
        print(json_file)
        # Открываем файл и загружаем данные
        with open(json_file, 'rb') as f:
            result = chardet.detect(f.read())
            print(result)

        with open(json_file, 'r', encoding = result['encoding']) as f:
            data = json.load(f)

        # Извлечение данных и преобразование их в DataFrame
        rows = []
        for item in data[next(iter(data))]:  # Используем next(iter(data)) для получения первого ключа в словаре
            for subitem in item['data']:
                row = {'text': subitem['text']}
                if 'entity' in subitem:
                    row['entity'] = subitem['entity']
                else:
                    row['entity'] = None
                rows.append(row)

        df = pd.DataFrame(rows)

        # Добавляем столбец с интентом
        df['intent'] = os.path.basename(json_file).split('_')[1].split('.')[0]

        # Если имя файла начинается с 'validate_', сохраняем данные в списке validate_data
        if os.path.basename(json_file).startswith('validate_'):
            validate_data.append(df)
        else:  # Иначе сохраняем данные в списке train_data
            train_data.append(df)

# Объединяем все DataFrame в списке в один DataFrame
train_data = pd.concat(train_data, ignore_index=True)
validate_data = pd.concat(validate_data, ignore_index=True)

# Выводим первые 5 строк каждого DataFrame
print("Train data:")
print(train_data.head())
print("\nValidate data:")
print(validate_data.head())


/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchCreativeWork/train_SearchCreativeWork.json
{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}
/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchCreativeWork/validate_SearchCreativeWork.json
{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}
/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchCreativeWork/train_SearchCreativeWork_full.json
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchScreeningEvent/validate_SearchScreeningEvent.json
{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}
/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchScreeningEvent/train_SearchScreeningEvent_full.json
{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/SearchScreeningEvent/train_SearchScreeningEvent.json
{'encoding': 'utf-8', 'confidence': 0.938125, 'language': 

In [13]:
# import chardet

# # Путь к файлу JSON
# json_file = "/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/PlayMusic/train_PlayMusic_full.json"

# # Определение кодировки файла
# with open(json_file, 'rb') as f:
#     result = chardet.detect(f.read())

# print(result)


{'encoding': 'MacRoman', 'confidence': 0.7299664915977274, 'language': ''}


In [30]:
# Удаляем дубликаты из обучающих данных
train_data = train_data.drop_duplicates()

# Удаляем дубликаты из валидационных данных
validate_data = validate_data.drop_duplicates()


In [31]:
# Сохраняем обучающие данные
train_data.to_csv('/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/train_data.csv', index=False)

# Сохраняем валидационные данные
validate_data.to_csv('/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/validate_data.csv', index=False)


In [32]:
import pandas as pd

# Путь к файлу CSV
train_data_path = "/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/train_data.csv"
validate_data_path = "/content/drive/MyDrive/Colab Notebooks/SNIPS_dataset/validate_data.csv"

# Чтение данных из CSV-файлов
train_data = pd.read_csv(train_data_path)
validate_data = pd.read_csv(validate_data_path)

# Вывод первых 5 строк каждого DataFrame
print("Train data:")
print(train_data.head())
print("\nValidate data:")
print(validate_data.head())


Train data:
                       text       entity              intent
0  Please help me find the           NaN  SearchCreativeWork
1        Bloom: Remix Album  object_name  SearchCreativeWork
2                                    NaN  SearchCreativeWork
3                      song  object_type  SearchCreativeWork
4                         .          NaN  SearchCreativeWork

Validate data:
                  text       entity              intent
0    Wish to find the           NaN  SearchCreativeWork
1                movie  object_type  SearchCreativeWork
2                 the           NaN  SearchCreativeWork
3           Heart Beat  object_name  SearchCreativeWork
4  Please look up the           NaN  SearchCreativeWork
